In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
from nltk.stem import PorterStemmer
import re
from textblob import TextBlob
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from nltk import word_tokenize 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn import datasets, linear_model
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
warnings.filterwarnings('ignore')






In [18]:
df_train = pd.read_csv(r'C:\Users\lucki\Desktop\train.csv') 
test = pd.read_csv(r'C:\Users\lucki\Desktop\test.csv') 
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [20]:
cleanr = re.compile('https?://\S+|www\.\S+')
p=[]
for i in range(len(df_train)):
    p.append(re.sub(cleanr, '', df_train['text'].loc[i]))
df_train['sin url']=p


In [21]:


stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df_train['texto procesado'] =df_train['sin url'].apply(lambda x: clean_text(x))

In [ ]:
a=[]
x=df_train['texto procesado']
for i in range(len(df_train)):
    a.append(TextBlob(df_train.loc[i,'texto procesado']).split())

df_train['texto blob']=a


In [ ]:
def joiner(text):
    text = " ".join(text)
    return text

df_train['texto blob'] =df_train['texto blob'].apply(lambda x: joiner(x))

In [ ]:
df_train=df_train[['keyword','location','texto blob','target']]

In [ ]:
df_train['keyword']=df_train['keyword'].fillna(value='Sin Keyword')
df_train['location']=df_train['location'].fillna(value='Sin Location')

In [ ]:
df_train['location'] =df_train['location'].apply(lambda x: clean_text(x))
df_train['keyword'] =df_train['keyword'].apply(lambda x: clean_text(x))

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
max_features = 6000

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train['texto blob'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['texto blob'])

maxlen = 150
X = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_train['target']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
xg_reg.fit(X_train, y_train)

In [ ]:
preds = xg_reg.predict(X_test)
preds=preds>=0.5
preds=preds.astype(int)
preds

In [ ]:
print(f1_score(y_test,preds))

In [ ]:
regresion_logistica = LogisticRegression()
regresion_logistica.fit(X_train,y_train)

In [ ]:
prediccion = regresion_logistica.predict(X_test)    
prediccion

In [ ]:
print(f1_score(y_test,prediccion))

In [ ]:
test_text=test['text']

In [ ]:
test=test.reset_index()

In [ ]:
testt=test[['id','text']]

In [ ]:
cleanr = re.compile('https?://\S+|www\.\S+')
q=[]
for i in range(len(test)):
    q.append(re.sub(cleanr, '', test['text'].loc[i]))
test['sin url']=q


In [ ]:
test['texto procesado'] =test['sin url'].apply(lambda x: clean_text(x))

In [ ]:
l=[]
x=test['texto procesado']
for i in range(len(test)):
    l.append(TextBlob(test.loc[i,'texto procesado']).split())

test['texto blob']=l

In [ ]:
def joiner(text):
    text = " ".join(text)
    return text

test['texto blob'] =test['texto blob'].apply(lambda x: joiner(x))

In [ ]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(test['texto blob'])
list_tokenized_train = tokenizer.texts_to_sequences(test['texto blob'])
maxlen = 130
testsub = pad_sequences(list_tokenized_train, maxlen=maxlen)


In [ ]:
predtest = regresion_logistica.predict(testsub)    
predtest

In [ ]:
test['target']=predtest

In [ ]:
pp['target']=predtest

In [ ]:
pp=pp.set_index('id')

In [ ]:
ruta=(r'C:\Users\lucki\Desktop\final.csv') 
pp.to_csv(ruta)

In [ ]:

pp

In [ ]:
test

In [ ]:
df_train['keyword']=df_train['keyword'].fillna(value='Sin Keyword')
df_train['location']=df_train['location'].fillna(value='Sin Location')

In [ ]:
df_train['key loc']=df_train['location']+' ' + df_train['keyword']

In [ ]:
keyloc=df_train[['key loc','target']]
keyloc

In [ ]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(keyloc)
X_train5=vectorizer5.transform(keyloc)
X_test5=vectorizer5.transform(keyloc)
X_train5

In [ ]:
X=keyloc['key loc']
y=keyloc['target']
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(X_train,y_train_)
X_train=vectorizer5.transform(keyloc)
X_test5=vectorizer5.transform(keyloc)


In [5]:
local=df_train[['location','target']]

NameError: name 'df_train' is not defined

In [ ]:
local

In [ ]:
P=local['location']
L=local['target']

In [ ]:

P_train, P_test, L_train, L_test = train_test_split(P, L, test_size=0.20, random_state=0)

In [ ]:
cls=LogisticRegression(random_state=0,solver='liblinear')


In [ ]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]


vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(P_train)
X_train5=vectorizer5.transform(P_train)
X_test5=vectorizer5.transform(P_test)
X_train5

In [ ]:
local_train=P_train + L_train

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(P_train)
list_tokenized_train = tokenizer.texts_to_sequences(P_train)

maxlen = 130
X_t = P_train
y = L_train

In [37]:
nuevo=df_train[['texto blob','target']]
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train['texto blob'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['texto blob'])

maxlen = 150
X = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_train['target']

In [38]:
X_t, XX, y_t, yy = train_test_split(X, y, test_size=0.20, random_state=0)

In [44]:
>>> from sklearn.ensemble import AdaBoostRegressor
>>> from sklearn.datasets import make_regression
>>> X, y = make_regression(n_features=4, n_informative=2,
...                        random_state=0, shuffle=False)
>>> regr = AdaBoostRegressor(random_state=0, n_estimators=100)
>>> regr.fit(X_t, y_t)
AdaBoostRegressor(n_estimators=100, random_state=0)
>>> regr.score(XX, yy)

0.02091319941959424

In [49]:
ada=regr.predict(XX)
adapre=ada > 0.5

In [51]:
adaprediction=adapre.astype(int)

In [53]:
len(adaprediction)

1523

In [55]:
(print(f1_score(yy,adaprediction)))

0.4041745730550285
